# 샘플 02: OpenAI SDK 통합

이 노트북은 Microsoft Foundry Local과 Azure OpenAI를 지원하며, 스트리밍 응답과 적절한 오류 처리를 포함한 OpenAI Python SDK의 고급 통합을 보여줍니다.

## 개요

이 샘플은 다음을 보여줍니다:
- Foundry Local과 Azure OpenAI 간의 원활한 전환
- 사용자 경험을 향상시키는 스트리밍 채팅 완료
- FoundryLocalManager SDK의 적절한 사용
- 견고한 오류 처리 및 대체 메커니즘
- 프로덕션 준비된 코드 패턴


## 사전 요구 사항

- **Foundry Local**: 설치 및 실행 중 (로컬 추론용)
- **Python**: OpenAI SDK가 포함된 3.8 이상 버전
- **Azure OpenAI**: 유효한 엔드포인트와 API 키 (클라우드 추론용)

### 종속성 설치


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## 라이브러리 가져오기 및 설정


In [ ]:
import os
import sys
from openai import OpenAI
import time
from typing import Tuple

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, manual configuration will be used")

## 구성 옵션

적절한 환경 변수를 설정하여 Azure OpenAI(클라우드) 또는 Foundry Local(디바이스 내) 중에서 선택하세요.


### 옵션 1: Azure OpenAI 구성

Azure OpenAI 자격 증명을 설정하고 주석을 제거하세요:


In [ ]:
# Azure OpenAI Configuration
# Uncomment and set your actual values

# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"  # e.g., "gpt-4"

print("Azure OpenAI configuration ready (if credentials are set)")

### 옵션 2: Foundry 로컬 구성

로컬 추론을 위한 기본 설정:


In [ ]:
# Foundry Local Configuration (default)
FOUNDRY_MODEL = "phi-4-mini"  # Change to your preferred model
FOUNDRY_BASE_URL = "http://localhost:51211"
FOUNDRY_API_KEY = ""  # Usually empty for local

print(f"Foundry Local configuration ready with model: {FOUNDRY_MODEL}")

## 클라이언트 팩토리 함수

이 함수들은 설정에 따라 적합한 OpenAI 클라이언트를 생성합니다:


In [ ]:
def create_azure_client() -> Tuple[OpenAI, str]:
    """Create Azure OpenAI client."""
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if not azure_endpoint or not azure_api_key:
        raise ValueError("Azure OpenAI endpoint and API key are required")
    
    model = os.environ.get("MODEL", "your-deployment-name")
    client = OpenAI(
        base_url=f"{azure_endpoint}/openai",
        api_key=azure_api_key,
        default_query={"api-version": azure_api_version},
    )
    
    print(f"🌐 Azure OpenAI client created with model: {model}")
    return client, model


def create_foundry_client() -> Tuple[OpenAI, str]:
    """Create Foundry Local client with SDK management."""
    alias = FOUNDRY_MODEL
    
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            print(f"🔄 Initializing Foundry Local with model: {alias}...")
            manager = FoundryLocalManager(alias)
            model_info = manager.get_model_info(alias)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            
            print(f"✅ Foundry Local SDK initialized")
            print(f"   Endpoint: {manager.endpoint}")
            print(f"   Model: {model_info.id}")
            return client, model_info.id
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{FOUNDRY_BASE_URL}/v1",
        api_key=FOUNDRY_API_KEY
    )
    
    print(f"🔧 Manual Foundry Local configuration")
    print(f"   Endpoint: {FOUNDRY_BASE_URL}/v1")
    print(f"   Model: {alias}")
    return client, alias

## 클라이언트 초기화

이 코드는 Azure OpenAI를 사용할지 Foundry Local을 사용할지 자동으로 감지합니다:


In [ ]:
def initialize_client() -> Tuple[OpenAI, str, str]:
    """Initialize the appropriate OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    
    if azure_endpoint and azure_api_key:
        print("🌐 Azure OpenAI configuration detected")
        try:
            client, model = create_azure_client()
            return client, model, "azure"
        except Exception as e:
            print(f"❌ Azure OpenAI initialization failed: {e}")
            print("🔄 Falling back to Foundry Local...")
    
    # Use Foundry Local
    print("🏠 Using Foundry Local configuration")
    try:
        client, model = create_foundry_client()
        return client, model, "foundry"
    except Exception as e:
        print(f"❌ Foundry Local initialization failed: {e}")
        raise

# Initialize the client
print("Initializing OpenAI client...")
print("=" * 50)
client, model, provider = initialize_client()
print("=" * 50)
print(f"✅ Initialization complete! Using {provider} with model: {model}")

## 기본 채팅 완료

간단한 채팅 완료를 테스트하세요:


In [ ]:
def simple_chat(prompt: str, max_tokens: int = 150) -> str:
    """Send a simple chat message and get response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test basic chat
test_prompt = "Say hello from the SDK quickstart and explain what you are in one sentence."

print(f"👤 User: {test_prompt}")
print("\n🤖 Assistant:")
response = simple_chat(test_prompt)
print(response)

## 스트리밍 채팅 완료

더 나은 사용자 경험을 위해 스트리밍 응답을 시연합니다:


In [ ]:
def streaming_chat(prompt: str, max_tokens: int = 300) -> str:
    """Send a chat message with streaming response."""
    try:
        print("🤖 Assistant (streaming):")
        
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        error_msg = f"Error: {e}"
        print(error_msg)
        return error_msg

# Test streaming chat
streaming_prompt = "Explain the key benefits of using Microsoft Foundry Local for AI development. Include aspects like privacy, performance, and cost."

print(f"👤 User: {streaming_prompt}\n")
streaming_response = streaming_chat(streaming_prompt)

## 다중 회화

대화의 맥락을 유지하는 방법을 보여줍니다:


In [ ]:
class ConversationManager:
    """Manages multi-turn conversations with context."""
    
    def __init__(self, system_prompt: str = None):
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def send_message(self, user_message: str, max_tokens: int = 200) -> str:
        """Send a message and get response while maintaining context."""
        # Add user message to conversation
        self.messages.append({"role": "user", "content": user_message})
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=self.messages,
                max_tokens=max_tokens
            )
            
            assistant_message = response.choices[0].message.content
            
            # Add assistant response to conversation
            self.messages.append({"role": "assistant", "content": assistant_message})
            
            return assistant_message
        except Exception as e:
            return f"Error: {e}"
    
    def get_conversation_length(self) -> int:
        """Get the number of messages in the conversation."""
        return len(self.messages)

# Create conversation manager with system prompt
system_prompt = "You are a helpful AI assistant specialized in explaining AI and machine learning concepts. Be concise but informative."
conversation = ConversationManager(system_prompt)

# Multi-turn conversation example
conversation_turns = [
    "What is the difference between AI inference on-device vs in the cloud?",
    "Which approach is better for privacy?",
    "What about performance and latency considerations?"
]

for i, turn in enumerate(conversation_turns, 1):
    print(f"\n{'='*60}")
    print(f"Turn {i}")
    print(f"{'='*60}")
    print(f"👤 User: {turn}")
    
    response = conversation.send_message(turn)
    print(f"\n🤖 Assistant: {response}")

print(f"\n📊 Conversation summary: {conversation.get_conversation_length()} messages total")

## 성능 비교

다양한 시나리오에 대한 응답 시간을 비교하세요:


In [ ]:
def benchmark_response_time(prompt: str, iterations: int = 3) -> dict:
    """Benchmark response time for a given prompt."""
    times = []
    responses = []
    
    for i in range(iterations):
        start_time = time.time()
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50  # Keep responses short for timing
            )
            
            end_time = time.time()
            response_time = end_time - start_time
            
            times.append(response_time)
            responses.append(response.choices[0].message.content)
            
        except Exception as e:
            print(f"Error in iteration {i+1}: {e}")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        return {
            "average_time": avg_time,
            "min_time": min_time,
            "max_time": max_time,
            "all_times": times,
            "sample_response": responses[0] if responses else None
        }
    
    return {"error": "No successful responses"}

# Benchmark different types of prompts
benchmark_prompts = [
    "What is AI?",
    "Explain machine learning in simple terms.",
    "List 3 benefits of edge computing."
]

print(f"⏱️  Performance Benchmark ({provider} - {model})")
print("=" * 60)

for prompt in benchmark_prompts:
    print(f"\n📝 Prompt: '{prompt}'")
    results = benchmark_response_time(prompt)
    
    if "error" not in results:
        print(f"   ⏰ Average time: {results['average_time']:.2f}s")
        print(f"   ⚡ Fastest: {results['min_time']:.2f}s")
        print(f"   🐌 Slowest: {results['max_time']:.2f}s")
        print(f"   📄 Sample response: {results['sample_response'][:100]}...")
    else:
        print(f"   ❌ {results['error']}")

## 고급 구성 및 오류 처리

다양한 매개변수와 오류 시나리오를 테스트하세요:


In [ ]:
def test_different_parameters():
    """Test chat completions with different parameters."""
    prompt = "Write a creative short story about AI."
    
    # Test different temperature values
    temperatures = [0.1, 0.5, 0.9]
    
    for temp in temperatures:
        print(f"\n🌡️ Temperature: {temp}")
        print("-" * 30)
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=temp
            )
            
            print(f"Response: {response.choices[0].message.content[:150]}...")
            
        except Exception as e:
            print(f"Error with temperature {temp}: {e}")

test_different_parameters()

## 서비스 상태 점검

서비스 상태 및 기능에 대한 종합적인 점검:


In [ ]:
def comprehensive_health_check():
    """Perform comprehensive health check of the service."""
    print("🏥 Comprehensive Health Check")
    print("=" * 50)
    
    # 1. Check model listing
    try:
        models_response = client.models.list()
        available_models = [m.id for m in models_response.data]
        print(f"✅ Model listing: SUCCESS")
        print(f"   📋 Available models: {available_models}")
        
        if model in available_models:
            print(f"   ✅ Current model '{model}' is available")
        else:
            print(f"   ⚠️ Current model '{model}' not found in available models")
    except Exception as e:
        print(f"❌ Model listing: FAILED - {e}")
    
    # 2. Test basic completion
    try:
        test_response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Say 'Health check successful'"}],
            max_tokens=10
        )
        print(f"✅ Basic completion: SUCCESS")
        print(f"   💬 Response: {test_response.choices[0].message.content}")
    except Exception as e:
        print(f"❌ Basic completion: FAILED - {e}")
    
    # 3. Test streaming
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Count to 3"}],
            max_tokens=20,
            stream=True
        )
        
        stream_content = ""
        chunk_count = 0
        for chunk in stream:
            if chunk.choices[0].delta.content:
                stream_content += chunk.choices[0].delta.content
                chunk_count += 1
        
        print(f"✅ Streaming: SUCCESS")
        print(f"   📦 Chunks received: {chunk_count}")
        print(f"   💬 Streamed content: {stream_content.strip()}")
    except Exception as e:
        print(f"❌ Streaming: FAILED - {e}")
    
    # 4. Provider-specific information
    print(f"\n📊 Configuration Summary:")
    print(f"   🏢 Provider: {provider}")
    print(f"   🤖 Model: {model}")
    if provider == "foundry":
        print(f"   🏠 Foundry SDK Available: {FOUNDRY_SDK_AVAILABLE}")
        print(f"   🔗 Base URL: {FOUNDRY_BASE_URL}")
    elif provider == "azure":
        print(f"   🌐 Azure Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT', 'Not set')}")
        print(f"   🔑 API Version: {os.environ.get('AZURE_OPENAI_API_VERSION', 'Not set')}")

comprehensive_health_check()

## 인터랙티브 테스트

이 셀을 사용하여 직접 프롬프트를 인터랙티브하게 테스트해보세요:


In [ ]:
# Interactive testing - modify the prompt below
custom_prompt = "Explain the concept of 'edge AI' and why it's becoming important."
use_streaming = True  # Set to False for regular completion

print(f"👤 Custom Prompt: {custom_prompt}\n")

if use_streaming:
    custom_response = streaming_chat(custom_prompt, max_tokens=250)
else:
    custom_response = simple_chat(custom_prompt, max_tokens=250)
    print(f"🤖 Assistant: {custom_response}")

## 요약 및 다음 단계

이 노트북은 OpenAI SDK의 고급 통합을 다음과 같이 시연했습니다:

### ✅ 주요 기능

1. **다중 제공자 지원**: Azure OpenAI와 Foundry Local 간의 원활한 전환
2. **스트리밍 응답**: 실시간 토큰 생성으로 UX 개선
3. **대화 관리**: 컨텍스트를 활용한 다중 턴 대화
4. **성능 벤치마킹**: 응답 시간 측정 및 분석
5. **포괄적인 상태 점검**: 서비스 검증 및 진단
6. **오류 처리**: 견고한 오류 처리 및 대체 메커니즘

### 🏆 Foundry Local vs Azure OpenAI

| 항목 | Foundry Local | Azure OpenAI |
|------|---------------|--------------|
| **프라이버시** | ✅ 데이터가 로컬에 유지됨 | ⚠️ 데이터가 클라우드로 전송됨 |
| **지연 시간** | ✅ 낮음 (로컬 추론) | ⚠️ 높음 (네트워크 의존성) |
| **비용** | ✅ 무료 (하드웨어 이후) | 💰 토큰당 비용 발생 |
| **오프라인** | ✅ 오프라인에서 작동 | ❌ 인터넷 필요 |
| **모델 다양성** | ⚠️ 제한된 선택 | ✅ 전체 모델 접근 가능 |
| **확장성** | ⚠️ 하드웨어 의존적 | ✅ 무제한 확장 가능 |

### 🚀 다음 단계

- **샘플 04**: Chainlit 채팅 애플리케이션 구축
- **샘플 05**: 다중 에이전트 오케스트레이션 시스템
- **샘플 06**: 지능형 모델 라우팅
- **프로덕션 배포**: 확장 및 모니터링 고려사항

### 💡 모범 사례

1. **제공자 간 대체 메커니즘을 항상 구현**하세요
2. **긴 응답에는 스트리밍을 사용**하여 성능을 개선하세요
3. **프로덕션 애플리케이션에는 적절한 오류 처리를 구현**하세요
4. **다양한 제공자의 응답 시간과 비용을 모니터링**하세요
5. **특정 요구 사항에 따라 적합한 제공자를 선택**하세요
